In [1]:
import cv2
import numpy as np
import dlib
import playsound
import time
from gtts import gTTS
from datetime import datetime
import pyttsx3
import datetime
import speech_recognition as sr
import wikipedia
import webbrowser
import os
from tkinter import *


In [2]:
# DLIB settings

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#Speech Engine settings

engine=pyttsx3.init('sapi5')  #sapi5 is a prog of microsoft i.e. used to control voices
voices=engine.getProperty('voices')
engine.setProperty('voice',voices[1].id)



In [3]:
# Keyboard settings

keyboard = np.zeros((800,1500,3),np.uint8)
keys_set_1 = {0:"Q",1:"W",2:"E",3:"R",4:"T",5:"Y",6:"U",7:"I",8:"O",9:"P",
              10:"A",11:"S",12:"D",13:"F",14:"G",15:"H",16:"J",17:"K",18:"L",
              19:"Z",20:"X",21:"C",22:"V",23:"B",24:"N",25:"M",
              26:"|_____|",27:"SAY"}
board = np.zeros((500,500),np.uint8)
board[:] = 255



In [4]:
# FUNCTIONS USED =>
def play_audio(audio):
    playsound.playsound(audio)

   

def letter(letter_index,this_letter,letter_light):
    # Keys
    if (letter_index==0):
        x=0
        y=0
    elif (letter_index==1):
        x=150
        y=0
    elif (letter_index==2):
        x=300
        y=0
    elif (letter_index==3):
        x=450
        y=0
    elif (letter_index==4):
        x=600
        y=0
    elif (letter_index==5):
        x=750
        y=0
    elif (letter_index==6):
        x=900
        y=0
    elif (letter_index==7):
        x=1050
        y=0
    elif (letter_index==8):
        x=1200
        y=0
    elif (letter_index==9):
        x=1350
        y=0
    elif (letter_index==10):
        x=0+70
        y=200
    elif (letter_index==11):
        x=150+70
        y=200
    elif (letter_index==12):
        x=300+70
        y=200
    elif (letter_index==13):
        x=450+70
        y=200
    elif (letter_index==14):
        x=600+70
        y=200
    elif (letter_index==15):
        x=750+70
        y=200
    elif (letter_index==16):
        x=900+70
        y=200
    elif (letter_index==17):
        x=1050+70
        y=200
    elif (letter_index==18):
        x=1200+70
        y=200
    elif (letter_index==19):
        x=150
        y=400
    elif (letter_index==20):
        x=300
        y=400
    elif (letter_index==21):
        x=450
        y=400
    elif (letter_index==22):
        x=600
        y=400
    elif (letter_index==23):
        x=750
        y=400
    elif (letter_index==24):
        x=900
        y=400
    elif (letter_index==25):
        x=1050
        y=400
    elif (letter_index==26):
        x=250
        y=600
    elif (letter_index==27):
        x=850
        y=600
    
    height = 200
    thickness = 3
    if letter_index == 26:
        width = 600
    elif letter_index == 27:
        width = 350
    else:
        width = 150

    # Text Settings
    font_letters = cv2.FONT_HERSHEY_PLAIN
    text = this_letter
    font_scale = 10
    font_thickness = 4

    if letter_light is True:
        cv2.rectangle(keyboard,(x+thickness,y+thickness),(x+width-thickness,y+height-thickness),(255,255,255),-1)
    else: 
        cv2.rectangle(keyboard,(x+thickness,y+thickness),(x+width-thickness,y+height-thickness),(255,0,0),thickness)
    
    text_size = cv2.getTextSize(text,font_letters,font_scale,font_thickness)[0] #has value = ((104,92) , 50) , hence taking first part only,i.e,(104,92)
    width_text , height_text = text_size[0],text_size[1] #taking 104 and 92  from (104,92)
    text_x = int((width - width_text)/2) +x
    text_y = int((height + height_text)/2)+y

    cv2.putText(keyboard,text,(text_x,text_y),font_letters,font_scale,(255,0,0),font_thickness)


def midpoint(p1,p2):
    return int((p1.x+p2.x)/2),int((p1.y+p2.y)/2)

def get_blinking_ratio(eye_points,facial_landmarks):
        left_point = (facial_landmarks.part(eye_points[0]).x,facial_landmarks.part(eye_points[0]).y)
        right_point = (facial_landmarks.part(eye_points[3]).x,facial_landmarks.part(eye_points[3]).y)
        center_top = midpoint(facial_landmarks.part(eye_points[1]),facial_landmarks.part(eye_points[2]))
        center_bottom = midpoint(facial_landmarks.part(eye_points[5]),facial_landmarks.part(eye_points[4]))
        #hor_line = cv2.line(frame,left_point,right_point,(0,255,0),2)
        #ver_line = cv2.line(frame,center_top,center_bottom,(0,255,0),2)
        
        #Ecledian distance to find the dist bw center top and bottom to check eye blinking :-root (x1-x2 sqaure + y1-y2 square)
        hor_line_length = hypot((left_point[0]-right_point[0]),(left_point[1]-right_point[1]))
        ver_line_length = hypot((center_top[0]-center_bottom[0]),(center_top[1]-center_bottom[1]))+0.0001
        ratio = hor_line_length/ver_line_length
        return ratio
    
def get_gaze_ratio(eye_points,facial_landmarks):
        left_eye_region = np.array([(facial_landmarks.part(eye_points[0]).x,facial_landmarks.part(eye_points[0]).y),
                                    (facial_landmarks.part(eye_points[1]).x,facial_landmarks.part(eye_points[1]).y),
                                    (facial_landmarks.part(eye_points[2]).x,facial_landmarks.part(eye_points[2]).y),
                                    (facial_landmarks.part(eye_points[3]).x,facial_landmarks.part(eye_points[3]).y),
                                    (facial_landmarks.part(eye_points[4]).x,facial_landmarks.part(eye_points[4]).y),
                                    (facial_landmarks.part(eye_points[5]).x,facial_landmarks.part(eye_points[5]).y)],np.int32)
        #print(left_eye_region)
        #cv2.polylines(frame,[left_eye_region],True,(0,0,255),2)
        
        height,width,_=frame.shape
        mask = np.zeros((height,width),np.uint8)
        
        cv2.polylines(mask,[left_eye_region],True,255,2) # color should be 3 in rgb but in  mask only 0 or 1 , hence only 2 colors black or white
        cv2.fillPoly(mask,[left_eye_region],255)
        eye = cv2.bitwise_and(gray , gray , mask=mask)
        
        
        min_x = np.min(left_eye_region[:,0])
        max_x = np.max(left_eye_region[:,0])
        min_y = np.min(left_eye_region[:,1])
        max_y = np.max(left_eye_region[:,1])
        
        left_eye_cropped=eye[min_y:max_y,min_x:max_x]
        
        _,threshold_eye = cv2.threshold(left_eye_cropped,70,255,cv2.THRESH_BINARY)
        
        h,w= threshold_eye.shape
        left_side_threshold = threshold_eye[0:h,0:int(w/2)]
        left_side_white = cv2.countNonZero(left_side_threshold)
        
        right_side_threshold = threshold_eye[0:h,int(w/2):w]
        right_side_white = cv2.countNonZero(right_side_threshold)
#         if left_side_white == 0:
#             right
        
        gaze_ratio = left_side_white/((0.0001)+right_side_white)
        return gaze_ratio
def speak(text):
    tts  =  gTTS(text=text,lang="en")
    x = datetime.now()
    s=""+str(x)
    s1 = "year_"+s[:4] +"_"+s[5:7]+"_"+s[8:10]+"_time"+"_"+s[11:13]+"_"+s[14:16]+"_"+s[17:19]
    filename = s1+"_.mp3"
    tts.save(filename)
    playsound.playsound(filename)

def speak(audio):
    engine.say(audio)
    engine.runAndWait()

def wishMe():
    hour=int(datetime.datetime.now().hour)
    if hour >=0 and hour <12:
        speak("Good Morning  Sir")
    elif hour >=12 and hour<18:
        speak("Good Afternoon  Sir")            
    else:
        speak("Good Evening  Sir")

    speak("        I am JARVIS          Please tell How may I help you?")

def takeCommand():
    # it takes microphone input from user and return string output
    r=sr.Recognizer()
    with sr.Microphone() as source:
        print(' Listening...')
        r.pause_threshold=0.5 # max gap can be 1 sec while giving input else it will take input after 1 sec gap
        audio=r.listen(source)
    try:
        print('Recognising...')
        query=r.recognize_google(audio,language="en-in")
        print('User said:-\n',query)
        
    except Exception as e:
        # print(e)
        print('Can you repeat it , PLEASE')
        return "None"
    return query


In [5]:
from math import hypot

In [6]:
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX


#Counters
frames_cnt = 0
letter_number = 0
blinking_frame = 0
text = " "     #single blank becoz it if by mistake as start SAY button pressed then also error
keyboard_selected = "left"
last_keyboard_selected = "left"
turn_cnt_left = 0 
turn_cnt_right = 0



t=Tk()

canvas=Canvas(t,width=300,height=160)

canvas.pack()

t.geometry("400x400")

var1 = StringVar()
label = Label( t, width=65, padx=10, pady=20, anchor=CENTER, font=88, textvariable=var1 )

var1.set("DIVYANG HELPER\nAn Aid to the Needy...")
label.pack()

var2 = StringVar()
label = Label( t, width=20, padx=15, pady=22, anchor=CENTER, font=24, textvariable=var2 )

var2.set("Select Your Choice\n1:Eye Keyboard\n2:Speech Recognition system")
label.pack()

def button_command():
    global choices
    choices = entry1.get()
    exit()
    #print("Inside func",choices)
    t.quit()
    return None 

entry1 = Entry(t,width = 40)
entry1.pack(padx = 20, pady = 20)
button1 = Button( text="Click", padx="66px", pady="20px", font="45", bg="lightblue", activeforeground="crimson", activebackground="red", command=button_command)
button1.pack()

t.mainloop()

print("New updated text is:-")
print(choices)


if choices != "1" and choices!="2":
    print("Wrong choice")
elif choices =="1":
    while True:
        ret,frame = cap.read(0)
        frame = cv2.resize(frame,None,fy=0.5,fx=0.5)
        keyboard[:] = (0,0,0) # this is used as without it keyboard key once lightened is always lightened , which is not desired
        frames_cnt+=1
        #showing directions
        new_frame = np.zeros((500,500,3),np.uint8)
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        active_letter = keys_set_1[letter_number]


        faces=detector(gray)

        for face in faces:
            x,y = face.left(),face.top()
            x1,y1 = face.right(),face.bottom()
            #cv2.rectangle(frame,(x,y),(x1,y1),(0,255,0),2)

            landmarks = predictor(gray,face)

            #DETECT BLINKING
            left_eye_ratio= get_blinking_ratio([36,37,38,39,40,41],landmarks)
            right_eye_ratio = get_blinking_ratio([42,43,44,45,46,47],landmarks)
            blinking_ratio = (right_eye_ratio+left_eye_ratio)/2

            if blinking_ratio >5.5:
                cv2.putText(frame,"Blinking",(50,55),font,2,(20,240,90),2)
                blinking_frame+=1
                frames_cnt -=1  # when eyes are closed dont let frames go ahead else letter would also change
                #Typing letters
                if blinking_frame == 7:
                    if active_letter == "|_____|":
                        text+=" "                  
                    elif active_letter == "SAY":
                        #speak(text)
                        print(text)
                        #below code to skip SAY button been pressed again(due to permission issue) by mistake in process of speaking
                        if last_keyboard_selected == "right":
                             letter_number+=1
                        elif last_keyboard_selected == "left":
                            letter_number-=1
                        letter_number%=28

                    else:
                        text+=active_letter
                        play_audio("pop.mp3")
                        time.sleep(1)
            else:
                blinking_frame = 0 # To stop count if we open the eyes

            #GAZE DETECTION

            gaze_ratio_left_eye = get_gaze_ratio([36,37,38,39,40,41],landmarks)
            gaze_ratio_right_eye = get_gaze_ratio([42,43,44,45,46,47],landmarks)

            gaze = (gaze_ratio_left_eye+gaze_ratio_right_eye)/2



            #using only gaze for right eye as it is giving more acccurate ressults than left

            side=""

            if blinking_frame > 0: # just to stop left right movement by mistake when eyes are closed
                continue
            elif blinking_frame == 0:
                if gaze<0.5:
                    side+="right"
                    new_frame[:] = (0,0,255)
                    keyboard_selected = "right"
                    #if keyboard_selected != last_keyboard_selected:

                    if  turn_cnt_left == 0:
                        turn_cnt_right+=1
                    else:
                        turn_cnt_left=0
                    if turn_cnt_right ==35:
                        play_audio("right.mp3")    
                        last_keyboard_selected = "right"

                elif gaze_ratio_left_eye>1.2:
                    side+="left"
                    new_frame[:] = (255,0,0)
                    keyboard_selected = "left"
                    #if keyboard_selected != last_keyboard_selected:
                    if turn_cnt_right == 0:
                        turn_cnt_left+=1
                    else:
                        turn_cnt_right=0
                    if turn_cnt_left == 40:
                        play_audio("left.mp3")
                        last_keyboard_selected = "left"
                else:
                     side+="center"



            cv2.putText(frame,str(gaze),(50,100),font,2,(0,0,255),3)

        # Drawing letters

        if frames_cnt == 30:    # Changing each letter after these many frames
            if last_keyboard_selected == "right":
                 letter_number+=1
            elif last_keyboard_selected == "left":
                letter_number-=1
            letter_number%=28  # To Continue from start again even after the end of 15 letters
            frames_cnt=0
        for i in range(0,28):
            if i==letter_number:
                light = True
            else:
                light = False
            letter(i,keys_set_1[i],light)

        cv2.putText(board,text,(50,100),font,2,0,3)

        # Showing various frames    
        cv2.imshow("Frame = ",frame)
        cv2.imshow("New Frame = ",new_frame)
        cv2.imshow("Key board = ",keyboard)
        cv2.imshow("Board = ",board)
        key=cv2.waitKey(1)
        if key==27:
            break

    cap.release()
    cv2.destroyAllWindows()
elif choices == "2":
    #print("Yo")
    wishMe()
   

    if 1:
        query= takeCommand().lower()

        #logic for executing tasks based on query
        if 'wikipedia' in query:
            speak('Searching wikipedia...')
            query=query.replace('wikipedia','')
            results=wikipedia.summary(query,sentences=2)
            speak('According to wikipedia ')
            print(results)
            speak(results)

        elif 'open youtube' in query:
            webbrowser.open('youtube.com')

        elif 'open brainly' in query:
            webbrowser.open('brainly.com')

        elif 'open google' in query:
            webbrowser.open('google.com')

        elif 'open stackoverflow' in query:
            webbrowser.open('stakoverflow.com')

        elif 'play music' in query:
            music_dir='C:\\music_dir'
            songs=os.listdir(music_dir)
            os.startfile(os.path.join(music_dir,songs[0]))

        elif 'the time' in query:
            strTime =datetime.datetime.now().strftime("%H:%M:%S") 
            speak(f"sir , the time is {strTime}")
           

        elif 'open code' in query:
            codepath="C:\\Users\\HP\\AppData\\Local\\Programs\\Microsoft VS Code\\Code.exe"
            os.startfile(codepath)

        elif 'email to mom' in query:
            try:
                speak('What should i send sir ')
                content=takeCommand()
                to='sramlax72@gmail.com'
                sendemail(to, content)
                speak('sir, email has been sent')
            except Exception as e:
                print(e)
                speak("Sorry but the email was not sent")


New updated text is:-
1
